In [ ]:
import glob
import re
from concurrent.futures import ThreadPoolExecutor
from glob import glob

import cv2
import numpy as np
import pandas as pd
import pydicom as dicom
from keras import Model, Input
from keras.src import backend
from keras.src import layers
from keras.src.applications import imagenet_utils
from keras.src.layers import Dense
from keras.src.models import Functional
from keras.src.ops import operation_utils
from keras.src.utils import file_utils
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.src.applications.densenet import DenseNet201
from keras.src.callbacks import Callback
import gc

In [ ]:
base_path = '/home/toru/PycharmProjects/rsna-2024-lumbar-spine-degenerative-classification'
train_png_file_path = f'{base_path}/train_dataset_png'

label_coordinates_df = pd.read_csv(f'{base_path}/train_label_coordinates.csv')
train_series = pd.read_csv(f'{base_path}/train_series_descriptions.csv')
df_train = pd.read_csv(f'{base_path}/train.csv')
df_sub = pd.read_csv(f'{base_path}/sample_submission.csv')
test_series = pd.read_csv(f'{base_path}/test_series_descriptions.csv')

train_dicom = f'{base_path}/train_images'
test_dicom = f'{base_path}/test_images'

model_path = f'model.keras'

In [ ]:
# 一つのstudy_idにseries_id が3つ未満のstudy_idを探す
study_id_list = train_series['study_id'].unique()
for study_id in study_id_list:
    series_id_list = train_series[train_series['study_id'] == study_id][
        'series_description'].unique()

    if len(series_id_list) < 3:
        print(f'study_id : {study_id}, series_id : {series_id_list}')

In [ ]:
# パラメータ
STUDY_NUM = 1975

# 出力のラベル
NORMAL_MILD = 0
MODERATE = 1
SEVERE = 2

# 重症度一覧
SEVERITY_LIST = ['Normal/Mild', 'Moderate', 'Severe']

# dicomデータの種類
SERIES_DESCRIPTION_LIST = train_series['series_description'].unique().tolist()
# 学習用パラメータ
BATCH_SIZE = 16
EPOCHS = 200
TRAIN_RATIO = 0.8
VALID_RATIO = 0.1
TEST_RATIO = 0.1

# 入出力の形状 (DenseNet201の入力サイズ)
INPUT_WIDTH = 512
INPUT_HEIGHT = 512
INPUT_CHANNEL_DICT = {
    'Sagittal T1': 17,
    'Sagittal T2/STIR': 17,
    'Axial T2': 29
}
IN_CHANS = sum(INPUT_CHANNEL_DICT.values())
IMG_SIZE = [INPUT_WIDTH, INPUT_HEIGHT]

CONDITIONS = [
    'spinal_canal_stenosis',
    'left_neural_foraminal_narrowing',
    'right_neural_foraminal_narrowing',
    'left_subarticular_stenosis',
    'right_subarticular_stenosis'
]

LEVELS = [
    'l1_l2',
    'l2_l3',
    'l3_l4',
    'l4_l5',
    'l5_s1',
]

N_LABELS = 25
N_CLASSES = 3 * N_LABELS

data_filenames = ['train_data', 'valid_data', 'test_data']

In [ ]:
_target_name = "right_subarticular_stenosis_l5_s1"

print(f'train shape : {df_train.shape}')
print(df_train[_target_name].unique())

# それぞれの重症度の数がいくらあるか計算
print(f'Normal/Mild : {df_train[df_train[_target_name] == str("Normal/Mild")].shape[0]}')
print(f'Moderate : {df_train[df_train[_target_name] == str("Moderate")].shape[0]}')
print(f'Severe : {df_train[df_train[_target_name] == str("Severe")].shape[0]}')

print(
    f'Normal/Mild ratio : {df_train[df_train[_target_name] == str("Normal/Mild")].shape[0] / df_train.shape[0]}')
print(
    f'Moderate ratio : {df_train[df_train[_target_name] == str("Moderate")].shape[0] / df_train.shape[0]}')
print(
    f'Severe ratio : {df_train[df_train[_target_name] == str("Severe")].shape[0] / df_train.shape[0]}')


In [ ]:


def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]


def get_train_label(study_id):
    """
    study_id に対応するラベルを取得 (testはデータがないのでtrainのみ)
    """

    row = df_train[df_train['study_id'] == study_id].to_numpy()[0][1:].tolist()

    row_category = np.zeros((N_LABELS, 3))
    for i in range(N_LABELS):
        if row[i] == 'Normal/Mild':
            row_category[i][0] = 1
        elif row[i] == 'Moderate':
            row_category[i][1] = 1
        elif row[i] == 'Severe':
            row_category[i][2] = 1
    # 1次元配列に変換
    row_category = row_category.flatten()
    return row_category


def get_file_list(study_id, series_desc, file_kind='train'):
    """
    study_id に対応するファイルリストを取得
    """
    dicom_file_list = []
    df = None
    if file_kind == 'train':
        df = train_series
    elif file_kind == 'test':
        df = test_series

    # study_id に対応する series_desc のリストを取得
    series_ids = df[df['study_id'] == study_id]
    series_id = series_ids[series_ids['series_description'] == series_desc]['series_id'].values
    if len(series_id) == 0:
        return dicom_file_list

    series_id = series_id[0]

    target_dicom_dir_path = ''
    if file_kind == 'train':
        target_dicom_dir_path = f'{train_dicom}/{study_id}/{series_id}/'
    elif file_kind == 'test':
        target_dicom_dir_path = f'{test_dicom}/{study_id}/{series_id}/'
    dicom_file_list = glob(target_dicom_dir_path + '/*.dcm')
    # ファイル名が数字になっているのでソート
    dicom_file_list = sorted(dicom_file_list, key=natural_keys)

    # series_desc にt対応するファイルリストを取得
    return dicom_file_list


def load_dicom_img(dicom_filepath):
    """
    dicomファイルを読み込む
    """
    dicom_data = dicom.read_file(dicom_filepath).pixel_array

    # 正規化
    dicom_data = (dicom_data - np.min(dicom_data)) / (np.max(dicom_data) - np.min(dicom_data) + 1e-8) * 255

    # 512x512にリサイズ
    dicom_data = cv2.resize(dicom_data, (INPUT_WIDTH, INPUT_HEIGHT), interpolation=cv2.INTER_CUBIC)

    # uint8 に変換
    dicom_data = dicom_data.astype(np.uint8)

    return dicom_data


def get_dicom_input_data(study_id, file_kind='train'):
    """
    study_id に対応するdicomデータを取得
    """

    input_dicom = np.zeros((INPUT_WIDTH, INPUT_HEIGHT, IN_CHANS))
    # Sagittal T1
    sagittal_t1_dicom_list = get_file_list(study_id, 'Sagittal T1', file_kind)
    if len(sagittal_t1_dicom_list) > 0:
        for i in range(INPUT_CHANNEL_DICT['Sagittal T1']):
            if i < len(sagittal_t1_dicom_list):
                dicom_data = load_dicom_img(sagittal_t1_dicom_list[i])
                input_dicom[:, :, i] = dicom_data

    # Sagittal T2/STIR
    sagittal_t2_dicom_list = get_file_list(study_id, 'Sagittal T2/STIR', file_kind)
    if len(sagittal_t2_dicom_list) > 0:
        for i in range(INPUT_CHANNEL_DICT['Sagittal T2/STIR']):
            if i < len(sagittal_t2_dicom_list):
                dicom_data = load_dicom_img(sagittal_t2_dicom_list[i])
                input_dicom[:, :, i + INPUT_CHANNEL_DICT['Sagittal T1']] = dicom_data

    # Axial T2
    axial_t2_dicom_list = get_file_list(study_id, 'Axial T2', file_kind)
    if len(axial_t2_dicom_list) > 0:
        for i in range(INPUT_CHANNEL_DICT['Axial T2']):
            if i < len(axial_t2_dicom_list):
                dicom_data = load_dicom_img(axial_t2_dicom_list[i])
                input_dicom[:, :, i + INPUT_CHANNEL_DICT['Sagittal T1'] + INPUT_CHANNEL_DICT[
                    'Sagittal T2/STIR']] = dicom_data

    # uint8 に変換
    input_dicom = input_dicom.astype(np.int8)
    return input_dicom


def get_dicom_and_label(study_id, file_kind):
    x_data = get_dicom_input_data(study_id, file_kind)
    y_data = get_train_label(study_id)
    return x_data, y_data


def generator(batch_size, study_ids, file_kind='train', is_augmentation=False, is_shuffle=False):
    """
    データセットの読み込み関数
    """
    while True:
        if is_shuffle:
            np.random.shuffle(study_ids)

        for start in range(0, len(study_ids), batch_size):
            gc.collect()

            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(study_ids))
            ids_train_batch = study_ids[start:end]

            with ThreadPoolExecutor() as executor:
                results = list(
                    executor.map(lambda sid: get_dicom_and_label(sid, file_kind), ids_train_batch))

            for x_data, y_data in results:
                if is_augmentation:
                    # 画像データにノイズを加える
                    x_data = x_data + np.random.normal(0, 0.1, x_data.shape)

                x_batch.append(x_data)
                y_batch.append(y_data)

            yield np.array(x_batch), np.array(y_batch)

def generator_load_all(batch_size, study_ids, file_kind='train', is_augmentation=False, is_shuffle=False):
    """
    実行時予めすべてのデータを読み込んだ上でデータを返す
    """
    x_batch = []
    y_batch = []
    print(f'事前にすべてのデータを読み込みます')
    
    with ThreadPoolExecutor() as executor:
        results = list(
            executor.map(lambda sid: get_dicom_and_label(sid, file_kind), study_ids))
    
    for x_data, y_data in results:
        x_batch.append(x_data)
        y_batch.append(y_data)
    
    print(f'データの読み込みが完了しました')
    # データをバッチサイズに分割して返す
    while True:
        for start in range(0, len(x_batch), batch_size):
            end = min(start + batch_size, len(study_ids))
            
            x_batch_batch = x_batch[start:end]
            y_batch_batch = y_batch[start:end]
            
            if is_augmentation:
                for i in range(len(x_batch_batch)):
                    # ノイズを加える
                    x_batch_batch[i] = x_batch_batch[i] + np.random.normal(0, 0.1, x_batch_batch[i].shape)
                    
            yield np.array(x_batch_batch), np.array(y_batch_batch)
            
def generate_and_save_data(study_ids, file_name, file_kind='train'):
    """
    学習用データを生成して保存
    """
    x_batch = []
    y_batch = []
    for sid in study_ids:
        print(f'残りのデータ数 : {len(study_ids) - len(x_batch)}')
        x_data, y_data = get_dicom_and_label(sid, file_kind)


        x_batch.append(x_data)
        y_batch.append(y_data)
    
    x_batch = np.array(x_batch)
    y_batch = np.array(y_batch)
    
    print(f'x_batch shape : {x_batch.shape}')
    print(f'y_batch shape : {y_batch.shape}')
    
    np.save(f'{file_name}_x.npy', x_batch)
    np.save(f'{file_name}_y.npy', y_batch)
    
    # メモリ解放
    del x_batch
    del y_batch
    gc.collect()
    
def save_train_data():
    """
    学習用データを保存
    """
    train_study_ids = get_train_study_id_array()
    generate_and_save_data(train_study_ids, file_name='train_data', file_kind='train')

def save_valid_data():
    """
    検証用データを保存
    """
    valid_study_ids = get_valid_study_id_array()
    generate_and_save_data(valid_study_ids, file_name='valid_data', file_kind='train')

def save_test_data():
    """
    テスト用データを保存
    """
    test_study_ids = get_test_study_id_array()
    generate_and_save_data(test_study_ids, file_name='test_data', file_kind='train')
        
# 学習用データの study_id を取得
def get_train_study_id_array():
    _study_ids = train_series['study_id'].unique()

    # 0.8 で分割
    train_study_ids = _study_ids[:int(len(_study_ids) * 0.8)]
    return train_study_ids


# 検証用データの study_id を取得
def get_valid_study_id_array():
    _study_ids = train_series['study_id'].unique()

    # 0.8 で分割
    valid_study_ids = _study_ids[int(len(_study_ids) * 0.8):int(len(_study_ids) * 0.9)]
    return valid_study_ids


# テスト用データの study_id を取得 (実際のテストデータではなく, 学習データの一部をテストデータとして使用)
def get_test_study_id_array():
    _study_ids = train_series['study_id'].unique()
    # 0.8 で分割
    test_study_ids = _study_ids[int(len(_study_ids) * 0.9):]
    return test_study_ids


def generator_for_test(batch_size, study_ids):
    """
    test_seriesからテスト用のデータセット作成
    kaggle で提出時に呼び出される
    :param batch_size: 
    :param study_ids: 
    :return: 
    """
    for start in range(0, len(study_ids), batch_size):
        x_batch = []
        end = min(start + batch_size, len(study_ids))
        ids_train_batch = study_ids[start:end]

        results = []
        for sid in ids_train_batch:
            results.append(get_dicom_input_data(sid, 'train'))
        for x_data in results:
            x_batch.append(x_data)

        yield np.array(x_batch), ids_train_batch
        
def load_data(file_name):
    """
    データを読み込む
    """
    x_data = np.load(f'{file_name}_x.npy')
    y_data = np.load(f'{file_name}_y.npy')
    
    return x_data, y_data

def generator_with_load_data(batch_size, 
                             study_ids, 
                             file_name,
                             file_kind='train', 
                             is_augmentation=False, 
                             is_shuffle=False):
    """
    事前にデータを読み込んでデータを返す
    """
    x_batch, y_batch = load_data(file_name)
    
    def shuffle_data_and_labels(x_batch, y_batch):
        # データとラベルをペアにしたリストを作成
        data_label_pairs = list(zip(x_batch, y_batch))
    
        # ペアのリストをシャッフル
        np.random.shuffle(data_label_pairs)
    
        # シャッフル後のデータとラベルをそれぞれ別々の配列に戻す
        shuffled_x_batch, shuffled_y_batch = zip(*data_label_pairs)
    
        # NumPy 配列に変換して返す（オプション）
        shuffled_x_batch = np.array(shuffled_x_batch)
        shuffled_y_batch = np.array(shuffled_y_batch)
    
        return shuffled_x_batch, shuffled_y_batch

    
    while True:
        # epochごとにシャッフル
        if is_shuffle:
            x_batch, y_batch = shuffle_data_and_labels(x_batch, y_batch)
            
        for start in range(0, len(x_batch), batch_size):
            end = min(start + batch_size, len(study_ids))
            
            x_batch_batch = x_batch[start:end]
            y_batch_batch = y_batch[start:end]
            
            if is_augmentation:
                for i in range(len(x_batch_batch)):
                    # ノイズを加える
                    x_batch_batch[i] = x_batch_batch[i] + np.random.normal(0, 0.1, x_batch_batch[i].shape)
                    
                    # ブラーをかける
                    x_batch_batch[i] = cv2.GaussianBlur(x_batch_batch[i], (5, 5), 0)
                    
            yield np.array(x_batch_batch), np.array(y_batch_batch)
        

In [ ]:
# 2492114990
for desc in SERIES_DESCRIPTION_LIST:
    dicom_image_list = get_file_list(2492114990, desc)
    print(f'{desc} : {len(dicom_image_list)}')

In [ ]:
# study_id のリストを作成
train_study_ids = get_train_study_id_array()
print(len(train_study_ids))
valid_study_ids = get_valid_study_id_array()
print(len(valid_study_ids))
test_study_ids = get_test_study_id_array()
print(len(test_study_ids))

In [ ]:
get_train_label(4003253)

In [ ]:
dicom_image_list = get_file_list(4003253, 'Sagittal T1')

input_image = get_dicom_input_data(4003253)
plt.imshow(input_image[:, :, 0:3])
input_data_gb = input_image.nbytes / 1024 / 1024 / 1024
print(f"画像サイズ[GB] : {input_data_gb}")
print(f'データセット全体のデータ数は {input_data_gb * STUDY_NUM}')

In [ ]:
# データセットの保存
save_train_data()
save_valid_data()
save_test_data()

In [ ]:
train_data_imgs, train_data_labels = load_data('train_data')

In [ ]:
train_data_imgs.shape